In [1]:
import torch, os
from torchvision import transforms
from torch.autograd import Variable
from torch.nn import functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import numpy as np
import cv2, torch
import numpy as np
import pandas as pd
import io
import requests
from PIL import Image
from collections import OrderedDict
from skimage import img_as_float,io

In [2]:
default_path= '/home/DL-based-Tumor-Classification/Data_mapped_images'
final_conv = 'conv3'
num_epochs = 50
batch_size = 400
learning_rate = 0.0001

Init Neural Net

In [3]:
class Net(nn.Module):

    def __init__(self, num_of_classes):
        super(Net, self).__init__()
        # input image channel, output channels, kernel size square convolution
        # kernel
        # input size = 102, output size = 100
        self.conv1 = nn.Conv2d(1, 64, kernel_size=5, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        # input size = 50, output size = 48
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        # input size = 24, output size = 24
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False)
        self.vp = nn.MaxPool2d(kernel_size=2, padding=0, stride=2)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(256*12*12, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, num_of_classes)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.bn1(self.vp(self.conv1(x))))
        x = F.relu(self.bn2(self.vp(self.conv2(x))))
        x = F.relu(self.bn3(self.vp(self.conv3(x))))
        x = self.drop2D(x)
        x = x.view(in_size, -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        return x


net = Net(num_of_classes=33)
net = net.double()
# net = nn.DataParallel(net)
net.cuda()

Net(
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop2D): Dropout2d(p=0.25, inplace=False)
  (vp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=36864, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=33, bias=True)
)

Load Pretrained Weight

In [4]:
def_path = default_path + '/img_fold8'
state_dict = torch.load(def_path + '/network_0505_th1.pth')
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
# load params
net.load_state_dict(new_state_dict)
net.eval()

RuntimeError: Error(s) in loading state_dict for Net:
	Missing key(s) in state_dict: "fc4.weight", "fc4.bias". 
	size mismatch for conv2.weight: copying a param with shape torch.Size([128, 64, 5, 5]) from checkpoint, the shape in current model is torch.Size([128, 64, 3, 3]).
	size mismatch for fc3.weight: copying a param with shape torch.Size([33, 512]) from checkpoint, the shape in current model is torch.Size([256, 512]).
	size mismatch for fc3.bias: copying a param with shape torch.Size([33]) from checkpoint, the shape in current model is torch.Size([256]).

Load Dataset

In [5]:
train_csv_path = def_path + '/train/labels_train.csv'
train_root_path = def_path + '/train'
test_csv_path = def_path + '/test/labels_test.csv'
test_root_path = def_path + '/test'
test_label_path =  def_path + '/test/test_label_run_th1_test.csv'
predicted_label_path = def_path + '/test/predicted_label_run_th1_test.csv'
model_path = def_path + '/network_0505_th1.pth'


class TumorDatasetTrain(Dataset):

    def __init__(self, csv_file , root_dir, transform=None):
        self.labels_frame = np.array(pd.read_csv(csv_file, skiprows=1, sep=',', header=None)).astype(np.int)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_frame)

    def __getitem__(self, idx):
        img_name = str(idx)+'.png'
        img_path = os.path.join(self.root_dir, img_name)
        """
            !!!Pay attention!!!
            The image size is set here
            """
        img = np.empty(shape=(1, 102, 102))
        img[0, :, :] = (img_as_float(io.imread(img_path)) - 0.5)/0.5
        label = np.array([self.labels_frame[idx,1]-1])
        train_sample = {'image': img, 'label': label}

        if self.transform:
            train_sample = self.transform(train_sample)
        return train_sample


class TumorDatasetTest(Dataset):

    def __init__(self, csv_file , root_dir, transform=None):
        self.labels_frame = np.array(pd.read_csv(csv_file, skiprows= 1, sep=',', header= None)).astype(np.int)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_frame)

    def __getitem__(self, idx):
        img_name = str(idx) + '.png'
        img_path = os.path.join(self.root_dir, img_name)
        """
            !!!Pay attention!!!
            The image size is set here
            """
        img = np.empty(shape=(1, 102, 102))
        img[0, :, :] = (img_as_float(io.imread(img_path)) - 0.5)/0.5
        label = np.array([self.labels_frame[idx, 1]-1])
        test_sample = {'image': img, 'label': label}

        if self.transform:
            test_sample = self.transform(test_sample)
        return test_sample


class ToTensor(object):

    def __call__(self, sample):
        image, labels = sample['image'], sample['label']
        return {'image': torch.from_numpy(image), 'label': torch.LongTensor(labels)}


train_dataset = TumorDatasetTrain(csv_file=train_csv_path, root_dir=train_root_path,
                                  transform=transforms.Compose([ToTensor()]))
test_dataset = TumorDatasetTest(csv_file=test_csv_path, root_dir=test_root_path, transform=transforms.Compose([ToTensor()]))
dataloader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=1)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)


In [6]:
# generate class activation mapping for the top1 prediction
def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    
#     print(feature_conv.shape)
    
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[class_idx].dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam

def get_cam(net, features_blobs, img_pil, classes, root_img, item):
    params = list(net.parameters())
    param_temp = np.array(params)
    
#     print(param_temp.shape)
    
    weight_softmax = np.squeeze(params[-2].data.cpu().numpy())
    
#     print('weight softmax')
#     print(weight_softmax.shape)
    
    normalize =  transforms.Normalize([0.5], [0.5])
    preprocess = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ])
#     img = np.empty(shape=(1, 102, 102))
#     img[0, :, :] = (img_as_float(io.imread(root_img)) - 0.5)/0.5
#     print(img.shape)
#     img_tensor = preprocess(img_pil)
#     img_variable = Variable(img_tensor).cuda()
    
    outputs = net(img_pil)
    
#     print('outputs')
#     print(outputs.shape)
    
    probs, idx = torch.max(outputs.data, 1)
    
#     print('probs')
#     print(probs.shape)
#     print('idx')
#     print(idx.shape)
    
    
    h_x = F.softmax(outputs, dim=1).data.squeeze()
    probs, idx = h_x.sort(0, True)
    
#     print("probs: " )
#     print('idx: ')
#     print("before numpy")
#     print(probs.shape)
#     print(idx.shape)

    
    probs = probs.cpu().numpy()
    idx = idx.cpu().numpy()
    
#     print("after numpy")
#     print(probs.shape)
#     print(idx.shape)

#     print('idx 0:' + str(idx[0]))
#     print(len(features_blobs))


    # output: the prediction
#     line = '{:.3f} -> {}'.format(probs[0], classes[idx[0]])
#     print(line)

    CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])

    # render the CAM and output
#     print('output CAM.jpg for the top1 prediction: %s' % classes[idx[0]])
    img = cv2.imread(root_img)
    height, width, _ = img.shape
    CAM = cv2.resize(CAMs[0], (width, height))
    heatmap = cv2.applyColorMap(CAM, cv2.COLORMAP_JET)
    result = heatmap * 0.3 + img * 0.5
    cv2.imwrite(default_path+ '/img_fold8' + '/coba4' + '/'+str(idx[0])+'_Cam_'+str(item)+'.jpg', result)
    print("DONE!")

In [7]:
features_blobs = []

def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

net._modules.get(final_conv).register_forward_hook(hook_feature)

for ii, test_sample in enumerate(dataloader_test):
        test_imgs = Variable(test_sample['image']).cuda()
        print("SEE THIS::: " , test_imgs.shape)
        test_label = Variable(test_sample['label']).cuda()
        print(test_label.shape)
        root = test_root_path + '/' + str(ii) + '.png'
        for i in range(0,test_imgs.size(0)):
            features_blobs = []
            print("now in loop "+str(i)+"!!!!!")
            test_imgs_temp = test_imgs[i,:,:,:]
            test_imgs_in = test_imgs_temp.unsqueeze(0)
            test_label_temp = test_label[i,:]
            test_label_in = test_label_temp.unsqueeze(0)
            get_cam(net, features_blobs, test_imgs_in, test_label_in, root, i)

SEE THIS:::  torch.Size([65, 1, 96, 96])
torch.Size([65, 1])
now in loop 0!!!!!
DONE!
now in loop 1!!!!!
DONE!
now in loop 2!!!!!
DONE!
now in loop 3!!!!!
DONE!
now in loop 4!!!!!
DONE!
now in loop 5!!!!!
DONE!
now in loop 6!!!!!
DONE!
now in loop 7!!!!!
DONE!
now in loop 8!!!!!
DONE!
now in loop 9!!!!!
DONE!
now in loop 10!!!!!
DONE!
now in loop 11!!!!!
DONE!
now in loop 12!!!!!
DONE!
now in loop 13!!!!!
DONE!
now in loop 14!!!!!
DONE!
now in loop 15!!!!!
DONE!
now in loop 16!!!!!
DONE!
now in loop 17!!!!!
DONE!
now in loop 18!!!!!
DONE!
now in loop 19!!!!!
DONE!
now in loop 20!!!!!
DONE!
now in loop 21!!!!!
DONE!
now in loop 22!!!!!
DONE!
now in loop 23!!!!!
DONE!
now in loop 24!!!!!
DONE!
now in loop 25!!!!!
DONE!
now in loop 26!!!!!
DONE!
now in loop 27!!!!!
DONE!
now in loop 28!!!!!
DONE!
now in loop 29!!!!!
DONE!
now in loop 30!!!!!
DONE!
now in loop 31!!!!!
DONE!
now in loop 32!!!!!
DONE!
now in loop 33!!!!!
DONE!
now in loop 34!!!!!
DONE!
now in loop 35!!!!!
DONE!
now in loop 3

In [36]:
print(test_label.size())
imgs = test_label[0,:]
print(imgs.shape)
print(imgs.unsqueeze(0).size())
unsq = imgs.unsqueeze(0)
print(unsq)

torch.Size([65, 1])
torch.Size([1])
torch.Size([1, 1])
tensor([[0]], device='cuda:0')


In [31]:
print(test_imgs.size(0)-1)

64
